### Problem Statement

### Import and add dependncy

In [ ]:
!pip install tensorflow opencv-python mediapipe scikit-learn matplotlib

In [1]:
import time
import numpy as np
from matplotlib import pyplot as plt
import os#Work with filepaths
import cv2#Open Cv
import mediapipe as mp
#To split data for training and testing 
from sklearn.model_selection import train_test_split
#To convert data into one encoded data
from tensorflow.keras.utils import to_categorical
#Importing sequential model,LSTM layer and dense layer
from tensorflow.keras.models import Sequential #Allows us to build sequential neural model
from tensorflow.keras.layers import LSTM,Dense
from tensorflow.keras.callbacks import TensorBoard
#tensorboard --logdir=. use this to check logs
#It allows us to logging inside tensorboard to trace and moniter our model as it is training
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

### Taking keypoints using mediapipe holistics

In [2]:
#Setting up mediapipe holistic's
#Media pipe holistic to make detection
#Media pipe drawing will draw those points
#We will create them as function to easily access them
mp_holistic=mp.solutions.holistic#Holistic model
mp_drawing=mp.solutions.drawing_utils#Drawing the utilites

In [3]:
type(mp_holistic)
mp_holistic.HandLandmark

<enum 'HandLandmark'>

In [4]:
#Creating mediapipe detection function
def mediapipe_detection(image,model):
    #To the function we pass image and holistic model for detection
    #So when wwe get feed from opencv it is of format bgr(blue,green,red)
    #For detection we need them to be rgb we will change that using opencv
    #Here image is the frames from opencv
    image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)#Color conversion
    image.flags.writeable=False#Image is no longer writeable
    results=model.process(image)#Making detection
    image=cv2.cvtColor(image,cv2.COLOR_RGB2BGR)#Color conversion
    image.flags.writeable=True#Image is writeable again
    return image,results #Returning the results

In [5]:
#To draw the points onto the image
def draw_landmarks(image,results):
    #drawing landmarks using mp_drawings.draw_landmark
    #For Face
    mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACEMESH_TESSELATION)
    #For Pose
    mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS)
    #For Left Hand
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS)
    #For Right Hand
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS)
    #Pose landmark shows what landmark connected to other landmarks

In [6]:
mp_holistic.FACEMESH_TESSELATION

frozenset({(18, 17),
           (82, 38),
           (8, 9),
           (456, 248),
           (167, 2),
           (303, 271),
           (69, 104),
           (253, 450),
           (41, 42),
           (315, 16),
           (73, 74),
           (339, 373),
           (258, 385),
           (105, 63),
           (219, 48),
           (304, 272),
           (236, 217),
           (231, 22),
           (85, 84),
           (365, 367),
           (237, 218),
           (243, 112),
           (436, 427),
           (409, 410),
           (248, 456),
           (330, 266),
           (136, 135),
           (88, 95),
           (394, 430),
           (107, 108),
           (118, 117),
           (439, 438),
           (442, 443),
           (260, 466),
           (115, 48),
           (342, 276),
           (121, 128),
           (200, 421),
           (113, 124),
           (132, 137),
           (23, 230),
           (343, 357),
           (443, 444),
           (62, 78),
           (26,

In [8]:
#Formatted draw style landmarks
def draw_styled_landmarks(image,result):
    #For Face
    mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACEMESH_TESSELATION
                              #color landmark
                             ,mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius=1)
                             #color connections
                             ,mp_drawing.DrawingSpec(color=(80,256,121),thickness=1,circle_radius=1))
    #For Pose
    mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS
                             #color landmark
                             ,mp_drawing.DrawingSpec(color=(80,22,10),thickness=2,circle_radius=4)
                             #color connections
                             ,mp_drawing.DrawingSpec(color=(80,44,121),thickness=2,circle_radius=2))
    #For Left Hand
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS
                             #color landmark
                             ,mp_drawing.DrawingSpec(color=(121,22,76),thickness=2,circle_radius=4)
                             #color connections
                             ,mp_drawing.DrawingSpec(color=(121,44,250),thickness=2,circle_radius=2))
    #For Right Hand
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS
                             #color landmark
                             ,mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=4)
                             #color connections
                             ,mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2))

In [9]:
#Accessing video through webcm using OpenCV
#We loop thorugh all frames in camera to create video
cap=cv2.VideoCapture(0)#To acccess our webcam 
#here 0 represents device

#Accessing the holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():#It checks wheather we are accessing or not
        #Read feed
        ret,frame=cap.read()#It reads our frames
        
        #Make detections
        image,results=mediapipe_detection(frame,holistic)
        print("frame",frame)
        print("image",image)
        #Drwing Landmarks
        draw_styled_landmarks(image,results)
        
        #To showw to screen
        #Rendering
        cv2.imshow("OpenCv Feed",image)

        #To Exit o break the feed
        if cv2.waitKey(10) & 0xFF==ord('q'):
            break #it waits and if we press q breaks the loop
    cap.release()#It releases the webcam
    cv2.destroyAllWindows()#Destroy the cv window

frame [[[ 78 108 146]
  [ 79 109 145]
  [ 81 110 144]
  ...
  [122 142 149]
  [124 143 150]
  [124 143 150]]

 [[ 79 110 146]
  [ 79 110 145]
  [ 79 110 144]
  ...
  [122 142 149]
  [123 142 149]
  [124 143 150]]

 [[ 79 110 146]
  [ 79 110 144]
  [ 79 111 142]
  ...
  [117 142 146]
  [121 143 146]
  [120 142 144]]

 ...

 [[200 180 136]
  [199 180 137]
  [199 181 139]
  ...
  [ 64  66  59]
  [ 64  67  55]
  [ 64  67  55]]

 [[198 178 137]
  [200 180 142]
  [199 179 145]
  ...
  [ 64  65  60]
  [ 64  66  57]
  [ 63  65  56]]

 [[198 178 137]
  [199 180 144]
  [197 179 148]
  ...
  [ 64  65  62]
  [ 63  64  59]
  [ 62  63  58]]]
image [[[ 78 108 146]
  [ 79 109 145]
  [ 81 110 144]
  ...
  [122 142 149]
  [124 143 150]
  [124 143 150]]

 [[ 79 110 146]
  [ 79 110 145]
  [ 79 110 144]
  ...
  [122 142 149]
  [123 142 149]
  [124 143 150]]

 [[ 79 110 146]
  [ 79 110 144]
  [ 79 111 142]
  ...
  [117 142 146]
  [121 143 146]
  [120 142 144]]

 ...

 [[200 180 136]
  [199 180 137]
  [199 1

frame [[[101 119 161]
  [101 119 161]
  [101 119 161]
  ...
  [117 146 163]
  [120 146 162]
  [120 146 162]]

 [[ 99 119 161]
  [100 119 161]
  [101 119 161]
  ...
  [119 145 162]
  [117 143 159]
  [118 144 160]]

 [[ 99 121 157]
  [ 99 120 158]
  [ 99 120 159]
  ...
  [118 140 158]
  [117 138 157]
  [117 138 157]]

 ...

 [[204 186 155]
  [203 186 155]
  [201 186 155]
  ...
  [ 86  81  81]
  [ 87  82  83]
  [ 86  81  81]]

 [[206 184 159]
  [205 185 157]
  [205 187 156]
  ...
  [ 87  82  83]
  [ 86  81  81]
  [ 86  81  81]]

 [[204 184 159]
  [204 184 159]
  [205 186 160]
  ...
  [ 87  83  79]
  [ 87  82  83]
  [ 86  81  81]]]
image [[[101 119 161]
  [101 119 161]
  [101 119 161]
  ...
  [117 146 163]
  [120 146 162]
  [120 146 162]]

 [[ 99 119 161]
  [100 119 161]
  [101 119 161]
  ...
  [119 145 162]
  [117 143 159]
  [118 144 160]]

 [[ 99 121 157]
  [ 99 120 158]
  [ 99 120 159]
  ...
  [118 140 158]
  [117 138 157]
  [117 138 157]]

 ...

 [[204 186 155]
  [203 186 155]
  [201 1

frame [[[ 92 121 164]
  [ 93 121 159]
  [ 95 122 156]
  ...
  [117 144 169]
  [114 145 169]
  [114 145 169]]

 [[ 91 120 163]
  [ 93 120 161]
  [ 95 120 159]
  ...
  [119 144 169]
  [116 145 169]
  [115 143 168]]

 [[ 99 119 163]
  [ 98 120 161]
  [ 96 119 158]
  ...
  [111 135 161]
  [109 135 162]
  [109 135 162]]

 ...

 [[199 184 149]
  [198 184 151]
  [198 185 154]
  ...
  [ 86  82  88]
  [ 88  84  92]
  [ 88  84  92]]

 [[200 183 148]
  [199 183 150]
  [200 185 154]
  ...
  [ 85  83  86]
  [ 86  86  88]
  [ 86  86  88]]

 [[200 183 148]
  [199 183 150]
  [200 185 154]
  ...
  [ 84  82  84]
  [ 85  84  87]
  [ 86  86  88]]]
image [[[ 92 121 164]
  [ 93 121 159]
  [ 95 122 156]
  ...
  [117 144 169]
  [114 145 169]
  [114 145 169]]

 [[ 91 120 163]
  [ 93 120 161]
  [ 95 120 159]
  ...
  [119 144 169]
  [116 145 169]
  [115 143 168]]

 [[ 99 119 163]
  [ 98 120 161]
  [ 96 119 158]
  ...
  [111 135 161]
  [109 135 162]
  [109 135 162]]

 ...

 [[199 184 149]
  [198 184 151]
  [198 1

frame [[[ 97 120 159]
  [ 97 119 158]
  [ 99 120 159]
  ...
  [113 142 158]
  [109 142 158]
  [109 142 158]]

 [[ 96 118 162]
  [ 97 118 162]
  [ 99 119 163]
  ...
  [114 141 159]
  [111 142 158]
  [111 142 158]]

 [[ 96 117 163]
  [ 97 118 162]
  [ 98 118 162]
  ...
  [111 133 155]
  [114 137 158]
  [114 137 158]]

 ...

 [[201 183 153]
  [202 184 151]
  [205 185 151]
  ...
  [ 99  96 109]
  [103 100 114]
  [108 105 119]]

 [[205 185 151]
  [203 184 149]
  [205 185 151]
  ...
  [ 97  97 106]
  [ 97 100 108]
  [103 105 114]]

 [[203 184 149]
  [205 185 151]
  [205 185 152]
  ...
  [ 96  94 104]
  [100  98 107]
  [103 101 111]]]
image [[[ 97 120 159]
  [ 97 119 158]
  [ 99 120 159]
  ...
  [113 142 158]
  [109 142 158]
  [109 142 158]]

 [[ 96 118 162]
  [ 97 118 162]
  [ 99 119 163]
  ...
  [114 141 159]
  [111 142 158]
  [111 142 158]]

 [[ 96 117 163]
  [ 97 118 162]
  [ 98 118 162]
  ...
  [111 133 155]
  [114 137 158]
  [114 137 158]]

 ...

 [[201 183 153]
  [202 184 151]
  [205 1

frame [[[ 90 115 158]
  [ 88 115 157]
  [ 86 116 156]
  ...
  [120 139 165]
  [119 140 168]
  [119 140 168]]

 [[ 90 115 158]
  [ 89 115 158]
  [ 88 115 158]
  ...
  [117 137 163]
  [119 138 163]
  [117 137 162]]

 [[ 88 115 158]
  [ 88 115 158]
  [ 88 115 158]
  ...
  [112 132 159]
  [111 132 158]
  [110 131 157]]

 ...

 [[204 184 159]
  [204 184 159]
  [204 184 159]
  ...
  [ 98  91 114]
  [104  96 122]
  [108 100 126]]

 [[201 185 159]
  [203 185 158]
  [204 185 157]
  ...
  [ 95  91 110]
  [ 96  92 111]
  [103  99 118]]

 [[204 184 159]
  [204 185 156]
  [204 186 154]
  ...
  [ 95  90 108]
  [ 97  92 106]
  [103  98 112]]]
image [[[ 90 115 158]
  [ 88 115 157]
  [ 86 116 156]
  ...
  [120 139 165]
  [119 140 168]
  [119 140 168]]

 [[ 90 115 158]
  [ 89 115 158]
  [ 88 115 158]
  ...
  [117 137 163]
  [119 138 163]
  [117 137 162]]

 [[ 88 115 158]
  [ 88 115 158]
  [ 88 115 158]
  ...
  [112 132 159]
  [111 132 158]
  [110 131 157]]

 ...

 [[204 184 159]
  [204 184 159]
  [204 1

...

frame [[[ 90 117 151]
  [ 88 116 154]
  [ 87 117 157]
  ...
  [129 147 172]
  [132 147 171]
  [132 147 171]]

 [[ 90 117 151]
  [ 90 116 154]
  [ 92 116 157]
  ...
  [129 147 172]
  [132 147 171]
  [132 147 171]]

 [[ 95 116 153]
  [ 93 117 155]
  [ 89 117 156]
  ...
  [125 145 170]
  [126 145 171]
  [126 145 171]]

 ...

 [[202 184 154]
  [200 185 154]
  [199 186 155]
  ...
  [ 90  88  96]
  [ 91  89  98]
  [ 91  89  98]]

 [[200 185 152]
  [200 185 153]
  [201 186 155]
  ...
  [ 91  88  94]
  [ 93  90  93]
  [ 93  90  93]]

 [[200 185 154]
  [201 184 154]
  [204 186 155]
  ...
  [ 91  88  95]
  [ 93  90  94]
  [ 93  90  94]]]
image [[[ 90 117 151]
  [ 88 116 154]
  [ 87 117 157]
  ...
  [129 147 172]
  [132 147 171]
  [132 147 171]]

 [[ 90 117 151]
  [ 90 116 154]
  [ 92 116 157]
  ...
  [129 147 172]
  [132 147 171]
  [132 147 171]]

 [[ 95 116 153]
  [ 93 117 155]
  [ 89 117 156]
  ...
  [125 145 170]
  [126 145 171]
  [126 145 171]]

 ...

 [[202 184 154]
  [200 185 154]
  [199 1

frame [[[ 93 115 152]
  [ 95 115 155]
  [ 93 113 157]
  ...
  [130 146 164]
  [136 146 162]
  [136 146 162]]

 [[ 94 117 156]
  [ 93 115 155]
  [ 93 115 156]
  ...
  [129 146 164]
  [134 147 162]
  [133 146 160]]

 [[ 98 118 162]
  [ 97 118 162]
  [ 94 116 159]
  ...
  [122 146 167]
  [122 145 166]
  [122 145 166]]

 ...

 [[201 182 157]
  [201 182 157]
  [204 184 159]
  ...
  [ 87  85  94]
  [ 88  86  96]
  [ 91  89  98]]

 [[201 182 157]
  [201 182 157]
  [204 184 159]
  ...
  [ 86  83  96]
  [ 87  84  98]
  [ 90  86 100]]

 [[201 183 155]
  [202 184 157]
  [204 184 159]
  ...
  [ 86  83  95]
  [ 87  84  98]
  [ 88  85  99]]]
image [[[ 93 115 152]
  [ 95 115 155]
  [ 93 113 157]
  ...
  [130 146 164]
  [136 146 162]
  [136 146 162]]

 [[ 94 117 156]
  [ 93 115 155]
  [ 93 115 156]
  ...
  [129 146 164]
  [134 147 162]
  [133 146 160]]

 [[ 98 118 162]
  [ 97 118 162]
  [ 94 116 159]
  ...
  [122 146 167]
  [122 145 166]
  [122 145 166]]

 ...

 [[201 182 157]
  [201 182 157]
  [204 1

frame [[[ 93 113 157]
  [ 95 113 156]
  [ 96 113 155]
  ...
  [129 147 173]
  [130 148 171]
  [130 148 171]]

 [[ 96 113 155]
  [ 96 115 156]
  [ 95 115 156]
  ...
  [128 147 173]
  [128 147 173]
  [128 147 173]]

 [[ 94 116 157]
  [ 95 116 157]
  [ 96 116 157]
  ...
  [127 147 173]
  [128 147 173]
  [126 145 171]]

 ...

 [[202 187 147]
  [201 185 152]
  [201 185 159]
  ...
  [ 86  85  92]
  [ 87  86  93]
  [ 88  87  94]]

 [[201 186 144]
  [201 185 151]
  [200 184 158]
  ...
  [ 86  85  92]
  [ 87  86  93]
  [ 87  86  93]]

 [[201 186 144]
  [200 185 148]
  [200 185 154]
  ...
  [ 86  85  91]
  [ 87  86  91]
  [ 87  86  91]]]
image [[[ 93 113 157]
  [ 95 113 156]
  [ 96 113 155]
  ...
  [129 147 173]
  [130 148 171]
  [130 148 171]]

 [[ 96 113 155]
  [ 96 115 156]
  [ 95 115 156]
  ...
  [128 147 173]
  [128 147 173]
  [128 147 173]]

 [[ 94 116 157]
  [ 95 116 157]
  [ 96 116 157]
  ...
  [127 147 173]
  [128 147 173]
  [126 145 171]]

 ...

 [[202 187 147]
  [201 185 152]
  [201 1

frame [[[ 89 116 159]
  [ 91 119 160]
  [ 90 120 158]
  ...
  [126 149 169]
  [127 150 170]
  [126 149 169]]

 [[ 92 118 162]
  [ 92 119 161]
  [ 92 119 160]
  ...
  [126 149 169]
  [126 149 169]
  [126 149 169]]

 [[ 92 119 160]
  [ 92 119 159]
  [ 92 119 158]
  ...
  [128 149 167]
  [128 147 162]
  [128 147 162]]

 ...

 [[198 184 156]
  [199 184 156]
  [200 184 156]
  ...
  [ 87  84  97]
  [ 88  85 101]
  [ 90  86 102]]

 [[198 184 158]
  [199 184 157]
  [200 184 156]
  ...
  [ 85  83  91]
  [ 88  86  96]
  [ 88  86  96]]

 [[194 184 158]
  [197 185 155]
  [200 185 152]
  ...
  [ 85  84  89]
  [ 87  86  93]
  [ 88  87  94]]]
image [[[ 89 116 159]
  [ 91 119 160]
  [ 90 120 158]
  ...
  [126 149 169]
  [127 150 170]
  [126 149 169]]

 [[ 92 118 162]
  [ 92 119 161]
  [ 92 119 160]
  ...
  [126 149 169]
  [126 149 169]
  [126 149 169]]

 [[ 92 119 160]
  [ 92 119 159]
  [ 92 119 158]
  ...
  [128 149 167]
  [128 147 162]
  [128 147 162]]

 ...

 [[198 184 156]
  [199 184 156]
  [200 1

frame [[[ 86 112 159]
  [ 86 113 156]
  [ 86 114 152]
  ...
  [121 138 163]
  [120 140 163]
  [120 140 163]]

 [[ 85 111 156]
  [ 87 112 156]
  [ 88 113 154]
  ...
  [120 137 161]
  [121 139 160]
  [122 140 161]]

 [[ 87 112 153]
  [ 88 112 153]
  [ 89 112 153]
  ...
  [116 135 163]
  [116 136 161]
  [117 137 162]]

 ...

 [[198 180 149]
  [197 180 150]
  [196 179 151]
  ...
  [ 86  83  90]
  [ 87  83  90]
  [ 88  84  92]]

 [[196 179 151]
  [194 178 151]
  [196 179 153]
  ...
  [ 89  83  89]
  [ 93  83  89]
  [ 93  83  89]]

 [[197 177 152]
  [197 177 152]
  [198 179 153]
  ...
  [ 87  83  91]
  [ 89  82  92]
  [ 89  82  92]]]
image [[[ 86 112 159]
  [ 86 113 156]
  [ 86 114 152]
  ...
  [121 138 163]
  [120 140 163]
  [120 140 163]]

 [[ 85 111 156]
  [ 87 112 156]
  [ 88 113 154]
  ...
  [120 137 161]
  [121 139 160]
  [122 140 161]]

 [[ 87 112 153]
  [ 88 112 153]
  [ 89 112 153]
  ...
  [116 135 163]
  [116 136 161]
  [117 137 162]]

 ...

 [[198 180 149]
  [197 180 150]
  [196 1

In [ ]:
draw_landmarks(frame,results)
plt.imshow(frame)
#This show last frame captured
#Color Conversion
plt.imshow(cv2.cvtColor(frame,cv2.COLOR_BGR2RGB))

In [ ]:
draw_styled_landmarks(frame,results)
plt.imshow(frame)
#This show last frame captured
#Color Conversion
plt.imshow(cv2.cvtColor(frame,cv2.COLOR_BGR2RGB))

### Extract Keypoints

In [10]:
#Obtaining values using list comprenhension
def extract_keypoints(results):
    #Pose
    # If else statement returns zero array if results is empty
    pose=np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    #We need to flatten it to get in one array
    #Left hand
    lh=np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(63)
    #Right hand
    rh=np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(63)
    #Face
    face=np.array([[res.x,res.y,res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
    #returning the values in a single concatenated array
    return np.concatenate([pose,face,lh,rh])

In [11]:
extract_keypoints(results).shape

(1662,)

### Setup folders for collection

In [12]:
#Path for exported data,Numpyarray
DATA_PATH=os.path.join('MP_data')

#Actions we are going to try to detect
actions=np.array(['hello','thanks','loveyou'])
#We use 30 different frames of data to detect actions
#30 videos worth of data
no_sequences=30
#Videos are going to be length of 30 frames
sequence_length=30

In [ ]:
#Creating folders to save the data
for action in actions:
    for sequence in range(no_sequences):
        try:
            #makedirs will create sub folders
            os.makedirs(os.path.join(DATA_PATH,action,str(sequence)))
        except:
            pass
#We will create 30 folders for every actions

### Collecting data for training and testing

In [ ]:
#Accessing video through webcm using OpenCV
#We loop thorugh all frames in camera to create video
cap=cv2.VideoCapture(0)#To acccess our webcam 
#here 0 represents device

#Accessing the holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    #Loop through actions:hello,thanks etc
    for action in actions:
        #Loop through videos each action 30 videos
        for sequence in range(no_sequences):
            #Loop through each frame per video 30 frames
            for frame_num in range(sequence_length):
                #Read feed
                ret,frame=cap.read()#It reads our frames

                #Make detections
                image,results=mediapipe_detection(frame,holistic)

                #Drwing Landmarks
                draw_styled_landmarks(image,results)
                    
                #Applying wait logic 
                #To give break between videos and say to user what video he is at
                if frame_num==0:
                    cv2.putText(image,'STARTING COLLECION',(120,200),
                                   cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),4,cv2.LINE_AA)
                    cv2.putText(image,f'Collecting frames for {action} Video number {sequence}'
                                    ,(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
                    cv2.waitKey(2000)#This waits for 2 second after every video captured
                    
                else:
                    cv2.putText(image,f'Collecting frames for {action} Video number {sequence}'
                                    ,(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)

                #Extracting the keypoints
                keypoints=extract_keypoints(results)
                #Saving the extracted keypoints
                #Path to save the key points
                npy_path=os.path.join(DATA_PATH,action,str(sequence),str(frame_num))
                #Saving keypoints
                np.save(npy_path,keypoints)
                    
                #To show to screen
                #Rendering
                cv2.imshow("OpenCv Feed",image)

                #To Exit o break the feed
                if cv2.waitKey(10) & 0xFF==ord('q'):
                    break #it waits and if we press q breaks the loop
    cap.release()#It releases the webcam
    cv2.destroyAllWindows()#Destroy the cv window

In [ ]:
#To destroy open cv in middle
cap.release()#It releases the webcam
cv2.destroyAllWindows()#Destroy the cv window

### Preprocessing Data and Creating Labels

In [13]:
#Creating label map
label_map={label:num for num,label in enumerate(actions)}

In [14]:
label_map
#We created dictionary for labels with set of id

{'hello': 0, 'thanks': 1, 'loveyou': 2}

In [15]:
#In preprocessing we put every frame np file in single one

In [16]:
#Creating 2 blank arrays
sequences,labels=[],[]
#here sequences represent our feature data(x) and labels represent labels(y)
#Going through our actions
for action in actions:
    #Going through our 30 videos
    for sequence in range(no_sequences):
        #Creating a blank array window
        window=[]
        #Going through each frames
        for frame_num in range(sequence_length):
            #Loading up the respective frame using np.load()
            res=np.load(os.path.join(DATA_PATH,action,str(sequence),f"{frame_num}.npy"))
            #Adding the value to the window array
            print(res)
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

[ 0.53316677  0.51246411 -1.15837407 ...  0.          0.
  0.        ]
[ 0.53625101  0.56368196 -1.19923615 ...  0.39377213  0.29293865
 -0.01883467]
[ 0.53835422  0.57232577 -1.19654667 ...  0.39214599  0.29512265
 -0.01889087]
[ 0.53891051  0.57860911 -1.19363427 ...  0.39359024  0.29590318
 -0.01519801]
[ 0.53914207  0.58111727 -1.1892035  ...  0.39277944  0.29678228
 -0.01560478]
[ 0.53749025  0.58275443 -1.28213179 ...  0.3922863   0.29380634
 -0.01977629]
[ 0.53303921  0.58212948 -1.29525244 ...  0.39375058  0.29127747
 -0.01606657]
[ 0.53024435  0.58285767 -1.27013588 ...  0.38737106  0.29633319
 -0.01552198]
[ 0.52779204  0.58106345 -1.31916416 ...  0.36694098  0.27612159
 -0.01156615]
[ 0.5271154   0.5809269  -1.30439985 ...  0.36535716  0.27915788
 -0.01530807]
[ 0.52289212  0.57910252 -1.28774858 ...  0.          0.
  0.        ]
[ 0.51913655  0.57603228 -1.27715421 ...  0.          0.
  0.        ]
[ 0.51505107  0.57465935 -1.27373374 ...  0.          0.
  0.        ]
[ 0.5

[ 0.45623225  0.68087822 -0.66326916 ...  0.          0.
  0.        ]
[ 0.45100564  0.68011379 -0.70767421 ...  0.          0.
  0.        ]
[ 0.44749361  0.67968041 -0.65306699 ...  0.          0.
  0.        ]
[ 0.4458217   0.67985898 -0.68309838 ...  0.          0.
  0.        ]
[ 0.44541794  0.6793474  -0.8443225  ...  0.          0.
  0.        ]
[ 0.44519949  0.67791301 -0.81260777 ...  0.          0.
  0.        ]
[ 0.44487554  0.67846352 -0.79270136 ...  0.          0.
  0.        ]
[ 0.44456768  0.67870861 -0.81170958 ...  0.          0.
  0.        ]
[ 0.44446415  0.6788969  -0.87396783 ...  0.          0.
  0.        ]
[ 0.44428587  0.67902112 -0.89748514 ...  0.          0.
  0.        ]
[ 0.44430241  0.67791247 -0.64261425 ...  0.          0.
  0.        ]
[ 0.44572949  0.67696667 -0.61244065 ...  0.          0.
  0.        ]
[ 0.447373    0.67679042 -0.60658151 ...  0.          0.
  0.        ]
[ 0.44814047  0.67634016 -0.6053977  ...  0.          0.
  0.        ]
[ 0.44

[ 0.39347667  0.56915867 -1.14004493 ...  0.          0.
  0.        ]
[ 0.39189294  0.56833369 -1.11958337 ...  0.          0.
  0.        ]
[ 0.39599466  0.56661832 -0.72563493 ...  0.          0.
  0.        ]
[ 0.39786765  0.56238002 -0.52107871 ...  0.08043747  0.83790904
 -0.00875162]
[ 0.39966634  0.56047195 -0.41489473 ...  0.08289254  0.95653886
  0.02577098]
[ 0.40046954  0.55629104 -0.40393853 ...  0.02033576  0.93625975
  0.00279302]
[ 0.40095073  0.5547657  -0.39576635 ... -0.00390439  0.90978944
  0.06033017]
[ 0.40113968  0.55360794 -0.38963655 ...  0.02676831  0.9535746
 -0.0270041 ]
[ 0.40036327  0.55374199 -0.41257352 ...  0.02125484  0.99472541
 -0.06521709]
[ 0.39770868  0.55373532 -0.40436068 ...  0.01487701  0.9734031
 -0.06396787]
[ 0.39653206  0.5534851  -0.45033422 ... -0.00479849  0.98499435
 -0.03241972]
[ 0.39487845  0.55346662 -0.41760039 ...  0.01054479  0.98184073
 -0.06354591]
[ 0.39442509  0.55343843 -0.40217972 ... -0.00783787  0.96151924
 -0.03898519]

[ 0.46588627  0.59280586 -0.79884857 ...  0.          0.
  0.        ]
[ 0.46589226  0.59240657 -0.8186776  ...  0.          0.
  0.        ]
[ 0.46444029  0.59207064 -0.80119991 ...  0.          0.
  0.        ]
[ 0.46345782  0.59180963 -0.79197174 ...  0.          0.
  0.        ]
[ 0.46198225  0.59180248 -0.78681427 ...  0.          0.
  0.        ]
[ 0.46310076  0.59286869 -0.87015516 ...  0.          0.
  0.        ]
[ 0.46378404  0.59353387 -0.89744663 ...  0.          0.
  0.        ]
[ 0.46620241  0.59353006 -0.72994411 ...  0.          0.
  0.        ]
[ 0.46816739  0.59351188 -0.71266091 ...  0.          0.
  0.        ]
[ 0.46790871  0.59349442 -0.61062181 ...  0.          0.
  0.        ]
[ 0.46770787  0.59344065 -0.58316267 ...  0.          0.
  0.        ]
[ 0.4665094   0.59335285 -0.61905134 ...  0.          0.
  0.        ]
[ 0.46625823  0.59337604 -0.64545828 ...  0.          0.
  0.        ]
[ 0.46485701  0.59346962 -0.67802906 ...  0.          0.
  0.        ]
[ 0.46

In [17]:
X=np.array(sequences)
X.shape

(90, 30, 1662)

In [18]:
y=to_categorical(labels).astype(int)
y
#Here 
#[1,0,0]-hello
#[0,1,0]-thanks
#[0,0,1]-bye

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0,

In [19]:
#Forming training and testing partition
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.04,random_state=1)
#4 for test other 86 to train

In [20]:
X_train.shape

(86, 30, 1662)

In [21]:
X_test.shape

(4, 30, 1662)

In [22]:
y_train.shape

(86, 3)

In [23]:
y_test.shape

(4, 3)

### Building and Training LSTM Neural Network

In [24]:
#Tensorboard allow us to moniter our accuracy as it is training
#Create a log directory to setuo tensorboard callbacks
log_dir=os.path.join('Logs')
tb_callback=TensorBoard(log_dir=log_dir)

In [25]:
#Initiating the model
model=Sequential()
#Adding 3 sets of LSTM models
model.add(LSTM(64,return_sequences=True,activation='relu',input_shape=(30,1662)))
#LSTM produces the hidden state and cell state for every timestep in the input data,
#thus preserving the temporal relationships between the input timesteps. This is particularly useful when stacking LSTM layers,
#where the second LSTM layer requires the sequence output from the first layer.
model.add(LSTM(128,return_sequences=True,activation='relu'))
model.add(LSTM(64,return_sequences=False,activation='relu'))
#Next layer is dense layer which we dont need to return sequences
#Take a look at andrew ng deeplearing specializations
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(actions.shape[0],activation='softmax'))

In [26]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['categorical_accuracy'])
#categorical_crossentropy is needed for multi classification model

In [ ]:
#Fit and train the model
model.fit(X_train,y_train,epochs=2000,callbacks=[tb_callback])

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 3)                 99        
                                                                 
Total params: 596675 (2.28 MB)
Trainable params: 596675 

### Making Prediction

In [29]:
res=model.predict(X_test)

1/1 [==============================] - 0s 310ms/step


In [30]:
for i in range(0,4):
    print(f"\nPrediction:{actions[np.argmax(res[i])]}\tActual Value:{actions[np.argmax(y_test[i])]}")


Prediction:thanks	Actual Value:thanks

Prediction:loveyou	Actual Value:loveyou

Prediction:thanks	Actual Value:thanks

Prediction:thanks	Actual Value:thanks


### Save Weights

In [ ]:
model.save('action.h5')

In [27]:
model.load_weights('action.h5')

### Evalution using confusion matrix and accuracy

In [31]:
yhat = model.predict(X_test)

1/1 [==============================] - 0s 21ms/step


In [32]:
#Extracting the predicted classes
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [33]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[1, 0],
        [0, 3]],

       [[3, 0],
        [0, 1]]], dtype=int64)

In [34]:
accuracy_score(ytrue, yhat)

1.0

### Testing in Realtime

In [35]:
#To render the probality
colors=[(245,117,16),(117,245,16),(16,117,245)]
#One color for each actions
def prob_viz(res,actions,input_frame,colors):
    output_frame=input_frame.copy()#Copy of the frames
    for num,prob in enumerate(res):
        #Dynamicllay placing the rectangle
        cv2.rectangle(output_frame,(0,60+num*40),(int(prob*100),90+num*40),colors[num],-1)
        cv2.putText(output_frame,actions[num],(0,85+num*40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
    return output_frame

In [ ]:
#New detection variables
sequence=[]#Collect 30 frames for prediction
sentence=[]#Concatenate our history of detection
threshold=0.7#To only give result if it is above threshold

#Accessing video through webcm using OpenCV
#We loop thorugh all frames in camera to create video
cap=cv2.VideoCapture(0)#To acccess our webcam 
#here 0 represents device

#Accessing the holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():#It checks wheather we are accessing or not
        #Read feed
        ret,frame=cap.read()#It reads our frames
        
        #Make detections
        image,results=mediapipe_detection(frame,holistic)
        
        #Drwing Landmarks
        draw_styled_landmarks(image,results)
        
        #Prediction Logic
        keypoints=extract_keypoints(results)
        sequence.append(keypoints)
        sequence=sequence[-30:]#Grabs last 30 frames
        
        #Run prediction if only 30 sequences collected
        if len(sequence)==30:
            res=model.predict(np.expand_dims(sequence,axis=0))[0]
            
            
        #Rendering logic to show prediction in the opencv feed
        #Checking wheather our result is above threshold
        if res[np.argmax(res)].any()>threshold:
            #Checking because we want to check next sequence
            if len(sentence)>0:
                #Checking if current action is not equal to last sentence
                if actions[np.argmax(res)]!=sentence[-1]:
                    sentence.append(actions[np.argmax(res)])
            else:
                sentence.append(actions[np.argmax(res)])
                
        if len(sentence)>5:
            #if sentence is greater than 5 grabbing last 5 value
            sentence=sentence[-5:]
            
        
        #Rendering to show the predection
        cv2.rectangle(image,(0,0),(640,40),(245,117,16),-1)
        cv2.putText(image,' '.join(sentence),(3,30),cv2.FONT_HERSHEY_SIMPLEX,
                       1,(255,255,255),2,cv2.LINE_AA)
        
        #Probabilities
        try:
            image=prob_viz(res,actions,image,colors)
        except TypeError:
            pass
        
        #To showw to screen
        #Rendering
        cv2.imshow("OpenCv Feed",image)

        #To Exit o break the feed
        if cv2.waitKey(10) & 0xFF==ord('q'):
            break #it waits and if we press q breaks the loop
    cap.release()#It releases the webcam
    cv2.destroyAllWindows()#Destroy the cv window

In [ ]:
#To destroy open cv in middle
cap.release()#It releases the webcam
cv2.destroyAllWindows()#Destroy the cv window